# Minecraft Paper Server on Google Colab

Run a high-performance Paper server for Minecraft. Set it up once, then start it anytime. Files are stored on Google Drive.

Source: [minecolab](https://github.com/devrock07/minecolab)

## Create Server (Run Once)

This script downloads Paper, sets up the server, and accepts the EULA.

In [ ]:
!curl ipinfo.io

In [ ]:
!curl ipinfo.io

In [ ]:
version = '1.21.4'  # Latest tested version. Others: 1.21, 1.20.6, 1.20.4
server_type = 'paper'  # Set to Paper

from google.colab import drive
import requests
import json
import os

drive.mount('/content/drive')
os.makedirs('/content/drive/My Drive/Minecraft-server', exist_ok=True)
os.chdir('/content/drive/My Drive/Minecraft-server')

def download_paper():
    url = f'https://api.papermc.io/v2/projects/paper/versions/{version}/builds'
    r = requests.get(url)
    if r.status_code != 200:
        print(f'Error: Paper version {version} not found.')
        return
    build = r.json()['builds'][-1]
    jar_url = f'{url}/{build}/downloads/paper-{version}-{build}.jar'
    print('Downloading Paper server...')
    r = requests.get(jar_url)
    if r.status_code == 200:
        with open('server.jar', 'wb') as f:
            f.write(r.content)
    else:
        print(f'Error downloading Paper. Status code: {r.status_code}')

download_paper()

with open('colabconfig.json', 'w') as f:
    json.dump({'server_type': server_type, 'server_version': version}, f)

with open('eula.txt', 'w') as f:
    f.write('eula=true')

print('Server setup complete! See Minecraft EULA at https://www.minecraft.net/en-us/eula')

## Start Server

Run this to launch your Paper server.

In [ ]:
import os
import json
from google.colab import drive

!sudo apt update -y &>/dev/null && echo 'APT updated.'
!sudo apt install openjdk-21-jre-headless -y &>/dev/null && echo 'Java 21 installed.'

java_ver = !java -version 2>&1 | awk -F[\"\.] -v OFS=. 'NR==1{print $2}'
print('Java 21 check: ' + ('OK' if java_ver[0] == '21' else f'Failed, using Java {java_ver[0]}. May reduce performance.'))

drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Minecraft-server')

colabconfig = json.load(open('colabconfig.json')) if os.path.exists('colabconfig.json') else {'server_type': 'paper'}

if not os.path.exists('server.jar'):
    print('Server JAR missing. Run the create server cell first.')
    exit()

server_flags = '-XX:+UseG1GC -XX:+ParallelRefProcEnabled -XX:MaxGCPauseMillis=200 -XX:+UnlockExperimentalVMOptions -XX:+AlwaysPreTouch -XX:G1HeapRegionSize=8M'
memory = '-Xms8704M -Xmx8704M'

region = 'in'
regions = ['us', 'eu', 'ap', 'au', 'sa', 'jp', 'in']
print(f'Default region: {region}. Options: {", ".join(regions)}')
if input('Change region? (yes/no): ').lower() == 'yes':
    new_region = input(f'Choose region ({", ".join(regions)}): ').lower()
    region = new_region if new_region in regions else region
    print(f'Region set to {region}.')

tunnel_service = 'playit'
print(f'Using tunnel: {tunnel_service}')

!java $memory $server_flags -jar server.jar nogui

## Notes

- Install the Playit.gg plugin for a static IP.
- Manage plugins and configs in Google Drive: `/content/drive/My Drive/Minecraft-server`.